In [1]:
import pandas as pd
import tensorflow as tf
import os
from PIL import Image
from PIL import ImageFile
from DogBreedIdentification_vgg16 import IMAGE_SIZE,inference_op

def _testdata_parse_function(filenames):
    features = {"img_raw": tf.FixedLenFeature([], tf.string)}
    parsed_features = tf.parse_single_example(filenames, features)
    image = tf.decode_raw(parsed_features['img_raw'], tf.uint8)
    return image

def predict_read_tfrecord(filename):
    read_tfrecord_dataset = tf.contrib.data.TFRecordDataset(filename)
    read_tfrecord_dataset = read_tfrecord_dataset.map(_testdata_parse_function)
    read_tfrecord_dataset = read_tfrecord_dataset.batch(1)
    return read_tfrecord_dataset

def testdata_convert_to_tfrecord(rootpath,  target_record_dir):
    if os.path.exists(target_record_dir) is True:
        print("the "+target_record_dir+" exist, no need to run the testdata_convert_to_tfrecord")
        return
    os.makedirs(target_record_dir, mode=0o775, exist_ok=True)
    filenames = os.listdir(rootpath)
    predict_id_list=[]
    for name in filenames:
        writer = tf.python_io.TFRecordWriter(target_record_dir+"/"+name+".tfrecords")
        print("Processing image:" + name)
        img = Image.open(rootpath+"/"+name)
        if img.mode == "RGB":
            img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
            img_raw = img.tobytes()
            example = tf.train.Example(
                features=tf.train.Features(feature={ 
                    "img_raw": tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))}))
            writer.write(example.SerializeToString())
        writer.close() 
    

In [2]:
if __name__ == '__main__':
    testdata_convert_to_tfrecord("./data/test", "./process_workspace/TestSet/")
    pre_image_holder = tf.placeholder(tf.float32, [1, IMAGE_SIZE, IMAGE_SIZE, 3])
    _, softmax, _, p = inference_op(pre_image_holder, 1)
    predict_result=[]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saverdict = {}
        for item in p:
            saverdict.update({item.name[:-2]: item})
        saver = tf.train.Saver(saverdict)
        if os.path.isfile("./process_workspace/DogBreedIdentificationVgg16_model.ckpt.meta") is True:
            print("Found checkpoint files , start to training from restored data")
            saver.restore(sess, "./process_workspace/DogBreedIdentificationVgg16_model.ckpt")
            print(p[0], sess.run(p[-1]))
            print("Model restored.\n")
        filenames = os.listdir("./process_workspace/TestSet")
        for file in filenames:
            testdataset = predict_read_tfrecord(["./process_workspace/TestSet/"+file])
            iterator = testdataset.make_initializable_iterator()
            testnext_element = iterator.get_next()
            sess.run(iterator.initializer)
            preprocess_testimg = sess.run(testnext_element)
            predict_softmax = sess.run([softmax], feed_dict={
                    pre_image_holder: preprocess_testimg[0].reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)/255-0.5
                })
            print("test img name:"+file)
            print(softmax)
            predict_result.append([file.rstrip(".jpg.tfrecords"),predict_softmax[0].tolist()])

the ./process_workspace/TestSet/ exist, no need to run the testdata_convert_to_tfrecord
Found checkpoint files , start to training from restored data
INFO:tensorflow:Restoring parameters from ./process_workspace/DogBreedIdentificationVgg16_model.ckpt
<tf.Variable 'conv1_1/w:0' shape=(3, 3, 3, 64) dtype=float32_ref> [-0.00129731 -0.00098393 -0.0023774  -0.00264734 -0.00202067 -0.00149379
 -0.00149654 -0.00160056 -0.00215215 -0.00270871 -0.00350231 -0.00260735
 -0.00203999 -0.00213429 -0.00302842 -0.00145519 -0.0009393  -0.00346301
 -0.00182204 -0.00191001 -0.0023635  -0.00241915 -0.00147808 -0.00208659
 -0.00265882 -0.00148558 -0.00227904 -0.00215934 -0.00277669 -0.00235836
 -0.00232352 -0.00165333 -0.00364198 -0.00150417 -0.00333861 -0.00311953
 -0.00207614 -0.00095401 -0.00306276 -0.00239172 -0.00197167 -0.00232535
 -0.00295241 -0.00143772 -0.00176017 -0.00232014 -0.00243433 -0.0018488
 -0.00208309 -0.00178257 -0.00244262 -0.00161635 -0.00268423 -0.00205626
 -0.00191509 -0.00313179 -0

TypeError: Fetch argument array([[ 0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.00833333,  0.00833333,  0.00833333,  0.00833333,
         0.00833333,  0.0